In [4]:
import json
import logging
import os
import sys

from dotenv import load_dotenv
from loguru import logger
import numpy as np
import pandas as pd
import s3fs
import sqlalchemy
from sqlalchemy import text

import geopandas as gpd
from shapely.geometry import LineString

sys.path.append('..')


import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
from dash.exceptions import PreventUpdate
from loguru import logger
import io
import pandas as pd
import datetime as dt
import calendar
import numpy as np

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)

project_path = 's3://sfgdata/projects/sustainable-transport'
dataset_name = 'sql_poc2'

In [10]:
df = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv', low_memory=False)

In [11]:
df.TKNUM.value_counts()

CV01377011    267
CV01402030    252
CV01416306    251
CV01416307    247
CV01377010    243
             ... 
CS01350128      1
CV01532316      1
CV01362718      1
CV01363091      1
0001340475      1
Name: TKNUM, Length: 36672, dtype: int64

In [12]:
df[df.TKNUM == 'CV01377011']

,IdSysteme,TKNUM,SHTYP,SHTYPLib,TPLST,TPLSTLib,VSART,VSARTLib,VSBED,VSBEDLib,ROUTE,ABF,TDLNR,TDLNRLib,TSNUM,VSTEL,VSTELLib,ADRKNZZ,ADRNZ,KUNNZ,KUNNR,KUNNZLib,Street,PostCode,City,Region,Country,YYEXIDV,YYBTGEW,YYGEWEI,YYVOLUM,YYVOLEH,NbColis
31706,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,59,FR20,FR45-Standard,C,4.936750e+05,1000059279,1000051082,MAT ELEC - ECHIROLLES,13 RUE DE COMBOIRE,38130,ECHIROLLES,NaN,FR,633891103871822895,4061.000,G,0.045,M3,NaN
31713,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,32,FR20,FR45-Standard,D,9.011150e+09,1000058095,1000057742,COMPTOIR ELECTRIQUE DE SARREBOURG,ZONE ARIANE BUHL,57403,SARREBOURG CEDEX,NaN,FR,633891103871822628,5493.000,G,0.095,M3,NaN
31755,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,202,FR20,FR45-Standard,C,1.143070e+06,1500035830,1000055508,PUYBARET ANDRE SA- BRIVE,9030 AVENUE CYPRIEN FAURIE,19100,BRIVE,NaN,FR,633891103871824325,14828.000,G,0.054,M3,NaN
31829,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,214,FR20,FR45-Standard,D,9.010916e+09,1500074150,1000058260,Thierry Verron,37 rue Helene Muller - Site de SA31 livraison ...,94320,THIAIS,NaN,FR,633891103871824448,17.574,KG,0.232,M3,NaN
31830,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,254,FR20,FR45-Standard,D,9.011211e+09,1000057912,1000054432,DISTRILEC CHOLET,"4, RUE BLERIOT",49300,CHOLET,NaN,FR,633891103871824844,24478.000,G,0.275,M3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408179,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,305,FR20,FR45-Standard,C,4.988490e+05,1000064388,CFR04830,PVELEC VIERZON,24 RUE CAMILLE DESMOULINS,18100,VIERZON,NaN,FR,633891103871825353,8567.000,G,0.106,M3,NaN
408274,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,310,FR20,FR45-Standard,C,4.893320e+05,1000054886,CFR04830,DIFFUSION ELECTRIQUE CORSE,ROUTE N 197,20256,CORBARA,NaN,FR,633891103871825407,7135.000,G,0.195,M3,NaN
408275,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,308,FR20,FR45-Standard,C,9.350090e+05,1500010644,CFR04830,CCL RODEZ,ROUTE D ESPALION,12850,ONET LE CHATEAU,NaN,FR,633891103871825384,14048.000,G,0.170,M3,NaN
408276,2,CV01377011,Z201,Black-BoxTS,FR45,DC Evreux,41,GRD Mono Parcel,NaN,NaN,FR5009,2022-05-19 16:21:00,10053492.0,DPD FRANCE,307,FR20,FR45-Standard,C,4.906690e+05,1000056314,CFR04830,REEL SAS AGENCE OUEST,ZI LES 4 VENTS NORD,50440,LA HAGUE,NaN,FR,633891103871825377,1903.000,G,0.010,M3,NaN


## Get and save SQL data

In [132]:
# Production forecasts provided to Finance
odbc_connect = (
    f"DRIVER={os.getenv('SQL_DRIVER')};" + 
    f"SERVER={os.getenv('SQL_SERVER')};" + 
    f"PORT=1433;" +
    f"DATABASE={os.getenv('SQL_DATABASE')};" +
    f"UID={os.getenv('USER_NAME')};" +
    f"PWD={os.getenv('PASSWORD')}"
)

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % odbc_connect)

In [333]:
filter_dc = ('HU07','FR45','FR43')
road_values = ('GRD groupage','GRD LTL','GRD FTL','Truck', 'Shuttle', 'GRD Mono Parcel')
europe_values = ('BE','EL','LT','PT','BG','ES','LU','RO','CZ','FR','HU','SI','DK','HR','MT','SK','DE','IT','NL','FI','EE','CY','AT','SE','IE','LV','PL')
filter_kunnz = 'ISNUMERIC([KUNNZ]) = 1'
query = f'SELECT * FROM .[Perf].[vwTRP_Transport] WHERE TPLST IN {filter_dc} AND VSARTLib IN {road_values} AND COUNTRY IN {europe_values} AND {filter_kunnz}'

with engine.begin() as conn:
    query = text(query)
    df = pd.read_sql_query(query, conn)

df.to_csv(f'{project_path}/raw/sql_poc2.csv', index=False)

In [334]:
df.columns

Index(['IdSysteme', 'TKNUM', 'SHTYP', 'SHTYPLib', 'TPLST', 'TPLSTLib', 'VSART',
       'VSARTLib', 'VSBED', 'VSBEDLib', 'ROUTE', 'ABF', 'TDLNR', 'TDLNRLib',
       'TSNUM', 'VSTEL', 'VSTELLib', 'ADRKNZZ', 'ADRNZ', 'KUNNZ', 'KUNNR',
       'KUNNZLib', 'Street', 'PostCode', 'City', 'Region', 'Country',
       'YYEXIDV', 'YYBTGEW', 'YYGEWEI', 'YYVOLUM', 'YYVOLEH', 'NbColis'],
      dtype='object')

In [346]:
pd.set_option('display.max_columns', None)
df[df['TKNUM']=='CV01213210']['YYGEWEI'].unique()
#266

array(['G', 'KG'], dtype=object)

In [133]:
filter_tplst = ('HU07','FR45','FR43')
filter_vsartlib = ('GRD groupage','GRD LTL','GRD FTL','Truck')
filter_country = ('BE','EL','LT','PT','BG','ES','LU','RO','CZ','FR','HU','SI','DK','HR','MT','SK','DE','IT','NL','FI','EE','CY','AT','SE','IE','LV','PL')
filter_kunnz = 'ISNUMERIC([KUNNZ]) = 1'
filter_date = "ABF BETWEEN '2022-10-01' AND '2022-10-31'"
query = f'SELECT * FROM .[Perf].[vwTRP_Transport]'

with engine.begin() as conn:
    query = text(query)
    df = pd.read_sql_query(query, conn)

In [134]:
df.head(20).to_csv('output/example.csv', index = False)

In [137]:
df.sample(50).to_excel('output/example.xlsx', index = False)

In [330]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report", minimal=True)
profile.to_file(f'output/sql_all_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [331]:
df.TPLST.unique()

array(['HU07', 'FR43', 'FRBG', 'BE44', 'FR45', 'FR1X', 'ZD00', 'BB00',
       'DK03', 'DK01'], dtype=object)

In [315]:
with engine.begin() as conn:
    columns_query = 'SELECT * FROM .[Perf].[vwTRP_Transport] FETCH FIRST 3 ROWS ONLY;'
    pd.read_sql(columns_query, conn)

ObjectNotExecutableError: Not an executable object: 'SELECT * FROM .[Perf].[vwTRP_Transport] FETCH FIRST 3 ROWS ONLY;'

In [15]:
df.to_csv(f'{project_path}/raw/sql_poc1.csv', index=False)

## EDA

In [281]:
df = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv')

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_7812\2391312233.py:1: DtypeWarning:

Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.



In [92]:
df.head()

,IdSysteme,TKNUM,SHTYP,SHTYPLib,TPLST,TPLSTLib,VSART,VSARTLib,VSBED,VSBEDLib,...,PostCode,City,Region,Country,YYEXIDV,YYBTGEW,YYGEWEI,YYVOLUM,YYVOLEH,NbColis
0,2,CS01178429,Z201,Black-BoxTS,FR43,DC Newlog,42,GRD LTL,01,Standard,...,91700,FLEURY MEROGIS,NaN,FR,636064800914202618,25.00,KG,0.250,M3,NaN
1,2,CS01390916,Z201,Black-BoxTS,FR43,DC Newlog,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,636064800930331682,3716.61,KG,33.978,M3,NaN
2,2,CV01673226,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884125105,63001.00,G,0.216,M3,NaN
3,2,CV01676298,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884287056,654444.00,G,5.616,M3,NaN
4,2,CV01677043,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884269533,180103.00,G,1.081,M3,NaN


In [288]:
df

,IdSysteme,TKNUM,SHTYP,SHTYPLib,TPLST,TPLSTLib,VSART,VSARTLib,VSBED,VSBEDLib,...,PostCode,City,Region,Country,YYEXIDV,YYBTGEW,YYGEWEI,YYVOLUM,YYVOLEH,NbColis
0,2,CS01178429,Z201,Black-BoxTS,FR43,DC Newlog,42,GRD LTL,01,Standard,...,91700,FLEURY MEROGIS,NaN,FR,636064800914202618,25.000,KG,0.250,M3,NaN
1,2,CS01390916,Z201,Black-BoxTS,FR43,DC Newlog,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,636064800930331682,3716.610,KG,33.978,M3,NaN
2,2,CV01673226,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884125105,63001.000,G,0.216,M3,NaN
3,2,CV01676298,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884287056,654444.000,G,5.616,M3,NaN
4,2,CV01677043,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,01,Standard,...,42150,LA RICAMARIE,NaN,FR,633891103884269533,180103.000,G,1.081,M3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165058,2,CS01172798,Z201,Black-BoxTS,FR43,DC Newlog,49,GRD groupage,NaN,NaN,...,59113,SECLIN,NaN,FR,636064800913822930,55.680,KG,3.384,M3,NaN
165059,2,CV01238697,Z201,Black-BoxTS,FR45,DC Evreux,42,GRD LTL,ZC,Parachevement Instn,...,38070,ST QUENTIN FALLAVIER CEDEX,NaN,FR,633891103866049658,585000.000,G,3.600,M3,NaN
165060,2,CS01259682,Z201,Black-BoxTS,FR43,DC Newlog,49,GRD groupage,NaN,NaN,...,35220,CHATEAUBOURG,NaN,FR,636064800920374101,64.465,KG,3.420,M3,NaN
165061,2,CS01273045,Z201,Black-BoxTS,FR43,DC Newlog,49,GRD groupage,NaN,NaN,...,69150,DECINES CHARPIEU,NaN,FR,636064800921279603,268.000,KG,1.728,M3,NaN


In [299]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df.sample(50000), title="Profiling Report")
profile.to_file(f'{project_path}/raw/{dataset_name}_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

OSError: [Errno 22] Invalid argument: 's3:\\sfgdata\\projects\\sustainable-transport\\raw\\sql_poc1_report.html'

In [95]:
df[df['Country']=='FR'].Region.unique()

array([nan, '60', '73', '69', '59', '57', '01', '34', '2B', '50', '77',
       '41', '93', '81', '92', '35', '56', '14', '68', '52', '51', '38',
       '70', '84', '42', '71', '67', '83', '91', '15', '95', '13', '33',
       '19', '62', '12', '66', '49', '76', '37', '02', '30', '32', '58',
       '08', '78', '10', '21', '72', '53', '28', '31', '26', '43', '82',
       '45', '06', '29', '47', '74', '63', '85', '44', '94', '16', '46',
       '07', '75', '22', '54', '40', '48', '64', '17', '18', '25', '80',
       '86', '79', '27', '65', '11', '87', '23', '03', '24', '89', '04'],
      dtype=object)

In [44]:
df[df.YYBTGEW==0.0]

,IdSysteme,TKNUM,SHTYP,SHTYPLib,TPLST,TPLSTLib,VSART,VSARTLib,VSBED,VSBEDLib,...,PostCode,City,Region,Country,YYEXIDV,YYBTGEW,YYGEWEI,YYVOLUM,YYVOLEH,NbColis
342,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,1220,#####,XXX,BG,659964851513694472,0.0,NaN,0.0,NaN,NaN
391,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,9000,#####,XXX,BG,659964851513694588,0.0,NaN,0.0,NaN,NaN
393,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,4004,#######,XXX,BG,659964851513694564,0.0,NaN,0.0,NaN,NaN
410,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,1000,#####,XXX,BG,659964851513694151,0.0,NaN,0.0,NaN,NaN
411,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,5800,######,PVN,BG,659964851513694373,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8901,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,5600,#####,XXX,BG,659964851513687818,0.0,NaN,0.0,NaN,NaN
9074,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,8000,######,XXX,BG,659964851513694434,0.0,NaN,0.0,NaN,NaN
9230,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,4000,#######,PDV,BG,659964851513694250,0.0,NaN,0.0,NaN,NaN
9493,2,CT00020618,Z201,Black-BoxTS,HU07,HU08-TPP RDC Cent.,49,GRD groupage,NaN,NaN,...,8800,######,SLV,BG,659964851513694519,0.0,NaN,0.0,NaN,NaN


array([10045939., 10047542., 10192267., 10060011., 10052112., 10014080.,
       10146678., 10041042., 10016074., 11087460.,       nan, 10009759.,
       10055990., 10007531.])

In [28]:
df.City.unique()

array(['FLEURY MEROGIS', 'LA RICAMARIE', 'MARSEILLE', ...,
       'CASTELNAU-MAGNOAC', 'CARCANS', 'VERNEUIL'], dtype=object)

In [32]:
df[['Street', 'City','PostCode', 'Country']].drop_duplicates()

,Street,City,PostCode,Country
0,RUE CLEMENT ADER,FLEURY MEROGIS,91700,FR
1,RUE REMI MOISE,LA RICAMARIE,42150,FR
6,11 AVENUE SEBASTIANELLI,MARSEILLE,13011,FR
9,LD FIGABRUNA,BIGUGLIA,20620,FR
11,3 RUE CLAUDE CHAPPE,FONDETTES,37230,FR
...,...,...,...,...
164890,4 Place des Combattants,CARCANS,33121,FR
164910,10 ALLEE DES ARTISANS,ANGLET,64600,FR
164995,483 AVENUE DES EAUX BLANCHES,SETE,34200,FR
165023,LD LES BARBIERS,VERNEUIL,58300,FR


In [58]:
df.shape

(165063, 33)

In [65]:
df_upper = df[['Street', 'City', 'PostCode', 'Country']]
for i in ['Street', 'City', 'Country']:
    df_upper[i] = df_upper[i].str.lower()
df_upper.drop_duplicates()

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_22044\4092548395.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upper[i] = df_upper[i].str.lower()


,Street,City,PostCode,Country
0,rue clement ader,fleury merogis,91700,fr
1,rue remi moise,la ricamarie,42150,fr
6,11 avenue sebastianelli,marseille,13011,fr
9,ld figabruna,biguglia,20620,fr
11,3 rue claude chappe,fondettes,37230,fr
...,...,...,...,...
164890,4 place des combattants,carcans,33121,fr
164910,10 allee des artisans,anglet,64600,fr
164995,483 avenue des eaux blanches,sete,34200,fr
165023,ld les barbiers,verneuil,58300,fr


In [20]:
df.TPLSTLib.value_counts()

DC Newlog             69132
HU08-TPP RDC Cent.    50861
DC Evreux             45070
Name: TPLSTLib, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165063 entries, 0 to 165062
Data columns (total 33 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   IdSysteme  165063 non-null  int64  
 1   TKNUM      165063 non-null  object 
 2   SHTYP      165063 non-null  object 
 3   SHTYPLib   165063 non-null  object 
 4   TPLST      165063 non-null  object 
 5   TPLSTLib   165063 non-null  object 
 6   VSART      165063 non-null  int64  
 7   VSARTLib   165063 non-null  object 
 8   VSBED      31664 non-null   object 
 9   VSBEDLib   31664 non-null   object 
 10  ROUTE      165063 non-null  object 
 11  ABF        165063 non-null  object 
 12  TDLNR      165062 non-null  float64
 13  TDLNRLib   165062 non-null  object 
 14  TSNUM      165063 non-null  int64  
 15  VSTEL      165063 non-null  object 
 16  VSTELLib   165063 non-null  object 
 17  ADRKNZZ    165063 non-null  object 
 18  ADRNZ      165063 non-null  int64  
 19  KUNNZ      165063 non-n

In [34]:
df.columns

Index(['IdSysteme', 'TKNUM', 'SHTYP', 'SHTYPLib', 'TPLST', 'TPLSTLib', 'VSART',
       'VSARTLib', 'VSBED', 'VSBEDLib', 'ROUTE', 'ABF', 'TDLNR', 'TDLNRLib',
       'TSNUM', 'VSTEL', 'VSTELLib', 'ADRKNZZ', 'ADRNZ', 'KUNNZ', 'KUNNR',
       'KUNNZLib', 'Street', 'PostCode', 'City', 'Region', 'Country',
       'YYEXIDV', 'YYBTGEW', 'YYGEWEI', 'YYVOLUM', 'YYVOLEH', 'NbColis'],
      dtype='object')

In [41]:
test = df.groupby(['KUNNZ', 'Street']).count().reset_index()

In [28]:
df.Street.value_counts()

CHEMIN DE CRUQUE PIGNON     2117
LIEU DIT LA GAULTIERE       1793
RUE MARCEL PAUL             1540
RUE CLEMENT ADER            1459
RUE BOBBY SANDS             1207
                            ... 
415 Rue de l'Etier             1
5 BIS  VOIE D?ANGLETERRE       1
25 RUE MICHEL COUET            1
3 AV LE CONCORDE               1
LD LES BARBIERS                1
Name: Street, Length: 8636, dtype: int64

## Get geolocation

In [54]:
df = pd.read_csv(f'{project_path}/processed/{dataset_name}_routes.csv')

In [51]:
df

,DC code,Client Nr,Address,City,Zip,Country,Country code
0,FR43,1500077814,RUE CLEMENT ADER,FLEURY MEROGIS,91700,France,FR
1,FR43,1500058266,RUE REMI MOISE,LA RICAMARIE,42150,France,FR
2,FR45,1500058266,RUE REMI MOISE,LA RICAMARIE,42150,France,FR
3,FR45,1000063871,11 AVENUE SEBASTIANELLI,MARSEILLE,13011,France,FR
4,FR45,1000068143,LD FIGABRUNA,BIGUGLIA,20620,France,FR
...,...,...,...,...,...,...,...
11902,FR43,1500067883,483 AVENUE DES EAUX BLANCHES,SETE,34200,France,FR
11903,FR43,1000056573,LD LES BARBIERS,VERNEUIL,58300,France,FR
11904,FR43,1500076628,43 Chemin de la Motte,VALENCE,26000,France,FR
11905,FR43,1000059122,15 RUE DU REC DE VEYRET,NARBONNE,11104,France,FR


In [55]:
# Get unique clients
df = df[['Client Nr','Address', 'City', 'Zip','Country', 'Country code']].drop_duplicates().reset_index(drop=True)

In [59]:
df.groupby(['Client Nr', 'Address']).sum().reset_index()['Client Nr'].value_counts()

1500077825    305
1500077290    292
1000066242    219
1500067883    201
1000058208    191
             ... 
1000060737      1
1000060749      1
1000060750      1
1000060757      1
1500142022      1
Name: Client Nr, Length: 4134, dtype: int64

In [61]:
df[df['Client Nr']==1500077825]

,Client Nr,Address,City,Zip,Country,Country code
32,1500077825,RUE CLEMENT ADER,FLEURY MEROGIS,91700,France,FR
164,1500077825,16 RUE DE L EUROPE,CHEMILLE EN ANJOU,49120,France,FR
176,1500077825,56 BOULEVARD NEY,PARIS,75018,France,FR
217,1500077825,61 RUE DU LANDY,AUBERVILLIERS,93300,France,FR
304,1500077825,SONEPAR CONNECT FONTENAY,FONTENAY SOUS BOIS,94120,France,FR
...,...,...,...,...,...,...
10175,1500077825,160 AVENUE ALBERT EINSTEIN,MOISSY CRAMAYEL,77550,France,FR
10177,1500077825,6 RUE DES PLATRIERES,MONTREUIL,93100,France,FR
10232,1500077825,30 RUE DU BUISSON AUX FRAISES,MASSY,91300,France,FR
10265,1500077825,137 RUE PROFESSEUR PAUL MILLIEZ,CHAMPIGNY SUR MARNE,94500,France,FR


In [47]:
df.Address.value_counts

<bound method IndexOpsMixin.value_counts of 0                   RUE CLEMENT ADER
1                     RUE REMI MOISE
2            11 AVENUE SEBASTIANELLI
3                       LD FIGABRUNA
4                3 RUE CLAUDE CHAPPE
                    ...             
9431         4 Place des Combattants
9432           10 ALLEE DES ARTISANS
9433    483 AVENUE DES EAUX BLANCHES
9434                 LD LES BARBIERS
9435         15 RUE DU REC DE VEYRET
Name: Address, Length: 9436, dtype: object>

## Compare different distance calculation methods using waypoints

In [143]:
# Import one AWS json file 
s3 = s3fs.S3FileSystem()
path = s3.ls(f'{project_path}/processed/aws')[0]
with s3.open(f's3://{path}', 'r') as fp:
        aws_json = json.load(fp)
line_string = aws_json['Geometry']['LineString']

print('Distance with AWS:', aws_json['Distance'])

Distance with AWS: 1349.63


In [132]:
import geopy.distance
sum = 0
for i in range(1,len(line_string)):
    x = geopy.distance.geodesic(line_string[i-1], line_string[i]).km
    sum = sum+x
print('Distance with Geopy: ', sum)

Distance with Geopy:  1875.5356448772409


In [140]:
from math import sin, cos, sqrt, atan2, radians
def get_distance(point1, point2):
    R = 6370
    lat1 = radians(point1[0])  #insert value
    lon1 = radians(point1[1])
    lat2 = radians(point2[0])
    lon2 = radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2- lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

sum = 0
for i in range(1,len(line_string)):
    x = get_distance(line_string[i-1], line_string[i])
    sum = sum+x
print('Distance with Geopy: ', sum)

# maybe some task missing?

Distance with Geopy:  1883.4989247775316


In [148]:
import requests
def get_distance(a, b):
    '''
    Getting distance with 
    '''

    # call the OSMR API
    r = requests.get(
        f"http://router.project-osrm.org/route/v1/car/{a[0]},{a[1]};{b[0]},{b[1]}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    try:
        route_1 = routes.get("routes")[0]
        return route_1['distance']
    except:
        return None

print('Distance with Bing:', get_distance(line_string[0], line_string[-1])/1000)

Distance with Bing: 1344.2806


In [ ]:
## to test: maybe with shapely?

In [147]:
from shapely.geometry import LineString
from shapely.ops import transform
from functools import partial
import pyproj

line1 = LineString(line_string)
print(str(line1.length) + " degrees")
# 0.0115488362184 degrees

# Geometry transform function based on pyproj.transform
project = partial(
    pyproj.transform,
    pyproj.Proj('EPSG:4326'),
    pyproj.Proj('EPSG:32633'))

line2 = transform(project, line1)
print(str(line2.length) + " meters")
# 1021.77585965 meters

16.978944879473474 degrees
2249060.586756993 meters


c:\Users\SESA667490\venvs\sustainable-transport\lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))


## Generated files

In [5]:
dataset_name = 'poc2'

In [1]:
import pandas as pd

In [6]:
df_raw = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv')

KeyboardInterrupt: 

In [144]:
start_geolocation_filepath = f'{project_path}/processed/df_shipper_geolocation_aws_modified.csv'
end_geolocation_filepath = f'{project_path}/processed/{dataset_name}_geolocation.csv'
routes_filepath = f'{project_path}/processed/{dataset_name}_routes.csv'
dict_column_names = 'bridge_names'

df_receiver = pd.read_csv(end_geolocation_filepath, dtype={'Zip': str}).dropna(subset=['Latitude']).rename(
    columns={"Longitude": "Receiver longitude", "Latitude": "Receiver latitude"})
df_shipper = pd.read_csv(start_geolocation_filepath, dtype={'Zip': str}).rename(
    columns={"Longitude": "Shipper longitude", "Latitude": "Shipper latitude"})
df_distances = pd.read_csv(f'{project_path}/processed/{dataset_name}_distances.csv')
df_raw = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv')

s3 = s3fs.S3FileSystem()
with s3.open(f'{project_path}/external/{dict_column_names}.json', 'r') as fp:
    dict_names = json.load(fp)

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_6780\1972863892.py:11: DtypeWarning:

Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.



In [50]:
dict_names = {
    "Pickup date": "ABF",
    "DC code": "TPLST",
    "DC name": "TPLSTLib",
    "Client no": "KUNNZ",
    "Client name": "KUNNZLib",
    "Address": "Street",
    "Zip": "PostCode",
    "City": "City",
    "Country code": "Country",
    "Shipping type": "VSART",
    "Carrier name": "TDLNRLib",
    "Sender weight (kg)": "YYBTGEW",
    "Volume (m3)": "YYVOLUM",
    "Nb of packages": "NbColis"
}